### Differs from anar.ipynb

* Used auto learning_rate reducer
* significantly increased accuracy (15% difference)

`Accuracy: 0.7750`

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import math
import sklearn

### Import and Prepare Data

In [2]:
training_data = pd.read_csv('/root/ubc/train.csv')

In [3]:
import pickle

with open('/root/ubc_ocean/anar/extracted-features/512px_resnet101_200.pkl', 'rb') as f:
    resnet101_200 = pickle.load(f)

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from collections import defaultdict
import random

# Set a fixed random state for reproducibility
random_state = 33

# Convert slide_features to a suitable format
data = [(features['features'], features['label']) for path, features in resnet101_200.items()]

# Organize data by labels
data_by_label = defaultdict(list)
for features, label in data:
    data_by_label[label].append((features, label))

# Split data for each label into train, validation, and test
train_data = []
val_data = []
test_data = []

for label, label_data in data_by_label.items():
    # Split data for this label into train and test with a fixed random state
    train_val_label_data, test_label_data = train_test_split(label_data, test_size=0.2, random_state=random_state)
    
    # Split train data into train and validation with a fixed random state
    train_label_data, val_label_data = train_test_split(train_val_label_data, test_size=0.125, random_state=random_state)  # 0.25 x 0.8 = 0.2 of original
    
    # Append split data to respective sets
    train_data.extend(train_label_data)
    val_data.extend(val_label_data)
    test_data.extend(test_label_data)

# Shuffle the datasets
random.seed(random_state)
random.shuffle(train_data)
random.shuffle(val_data)
random.shuffle(test_data)

# Function to check balance in each set
def check_balance(dataset):
    label_counts = defaultdict(int)
    for _, label in dataset:
        label_counts[label] += 1
    return dict(label_counts)

# Display balance of each set
print("Train balance:", check_balance(train_data))
print("Validation balance:", check_balance(val_data))
print("Test balance:", check_balance(test_data))

Train balance: {'CC': 28, 'EC': 28, 'LGSC': 28, 'HGSC': 28, 'MC': 28}
Validation balance: {'HGSC': 4, 'CC': 4, 'LGSC': 4, 'MC': 4, 'EC': 4}
Test balance: {'MC': 8, 'HGSC': 8, 'LGSC': 8, 'EC': 8, 'CC': 8}


In [5]:
# Create a mapping from label strings to integers
unique_labels = sorted(set(label for _, label in data))
label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}

class MILDataset(Dataset):
    def __init__(self, data, label_to_idx):
        self.data = data
        self.label_to_idx = label_to_idx

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        feature_vectors, label = self.data[idx]
        label_idx = self.label_to_idx[label]  # Convert label to integer
        return torch.tensor(feature_vectors), torch.tensor(label_idx, dtype=torch.float32)

# Create Datasets for train, validation, and test
train_dataset = MILDataset(train_data, label_to_idx)
val_dataset = MILDataset(val_data, label_to_idx)
test_dataset = MILDataset(test_data, label_to_idx)

# Create DataLoaders for each set
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

### Model Training

In [13]:
import torch.nn as nn
from sklearn.metrics import accuracy_score
from torch.optim.lr_scheduler import ReduceLROnPlateau

class AttentionMIL(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(AttentionMIL, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.attention = nn.Sequential(
            nn.Linear(hidden_dim, 1),
            nn.Softmax(dim=0)
        )
        self.classifier = nn.Linear(hidden_dim, num_classes)  # num_classes instead of 1

    def forward(self, bag):
        h = torch.relu(self.fc1(bag))
        a = self.attention(h)
        v = torch.sum(a * h, dim=0)
        y = self.classifier(v)  # Remove softmax here; output raw scores
        return y, a

# Number of unique classes
num_classes = len(unique_labels)

model = AttentionMIL(input_dim=2048, hidden_dim=256, num_classes=num_classes)
loss_function = nn.CrossEntropyLoss()  # CrossEntropyLoss for multiclass
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

# Early Stopping Parameters
best_val_loss = float('inf')
patience = 4
patience_counter = 0

# Model Training with Validation
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    # Training loop
    for bags, labels in train_loader:
        optimizer.zero_grad()
        bags = bags.squeeze(0)  # Remove the extra dimension from bags
        labels = labels.squeeze(0).long()  # Remove extra dimension and ensure long type for labels
        output, _ = model(bags)
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = torch.max(output.data, 0)
        train_total += 1
        train_correct += (predicted == labels).sum().item()

    train_accuracy = 100 * train_correct / train_total
    train_loss /= len(train_loader)

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for bags, labels in val_loader:
            bags = bags.squeeze(0)
            labels = labels.squeeze(0).long()
            output, _ = model(bags)
            loss = loss_function(output, labels)
            val_loss += loss.item()
            _, predicted = torch.max(output.data, 0)
            val_total += 1
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    val_loss /= len(val_loader)
    
    # Step the scheduler
    scheduler.step(val_loss)

    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Stopping early due to no improvement in validation loss.")
        break

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Validation Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')

Epoch 1/15, Train Loss: 1.6305, Train Acc: 18.57%, Validation Loss: 1.5788, Val Acc: 35.00%
Epoch 2/15, Train Loss: 1.5382, Train Acc: 35.00%, Validation Loss: 1.5200, Val Acc: 35.00%
Epoch 3/15, Train Loss: 1.3705, Train Acc: 42.86%, Validation Loss: 1.4352, Val Acc: 25.00%
Epoch 4/15, Train Loss: 1.2106, Train Acc: 53.57%, Validation Loss: 1.3701, Val Acc: 40.00%
Epoch 5/15, Train Loss: 1.0637, Train Acc: 57.86%, Validation Loss: 1.4177, Val Acc: 35.00%
Epoch 6/15, Train Loss: 0.9515, Train Acc: 67.14%, Validation Loss: 1.1763, Val Acc: 55.00%
Epoch 7/15, Train Loss: 0.8304, Train Acc: 67.86%, Validation Loss: 1.1639, Val Acc: 55.00%
Epoch 8/15, Train Loss: 0.7353, Train Acc: 72.86%, Validation Loss: 1.1676, Val Acc: 55.00%
Epoch 9/15, Train Loss: 0.6542, Train Acc: 74.29%, Validation Loss: 1.0692, Val Acc: 55.00%
Epoch 10/15, Train Loss: 0.5979, Train Acc: 76.43%, Validation Loss: 1.1712, Val Acc: 60.00%
Epoch 11/15, Train Loss: 0.5170, Train Acc: 81.43%, Validation Loss: 1.1743, Va

In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for bags, labels in test_loader:
        output, _ = model(bags.squeeze(0))
        _, predicted_labels = torch.max(output, 0)  # Get the index of the max log-probability
        predictions.append(predicted_labels.item())  # Append scalar value
        true_labels.append(labels.squeeze(0).item())  # Append scalar value

# Convert lists to arrays for metric calculation
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='macro', zero_division=1)
recall = recall_score(true_labels, predictions, average='macro')
f1 = f1_score(true_labels, predictions, average='macro')

# Print the metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.7750
Precision: 0.7648
Recall: 0.7750
F1 Score: 0.7608


In [15]:
idx_to_label = {idx: label for label, idx in label_to_idx.items()}

# Use idx_to_label to map numeric predictions back to label names
predicted_labels = [idx_to_label[int(idx)] for idx in predictions]
true_label_names = [idx_to_label[int(idx)] for idx in true_labels]

# Now predicted_labels and true_label_names contain the label names
print(predicted_labels)
print(true_label_names)

['MC', 'LGSC', 'EC', 'LGSC', 'LGSC', 'HGSC', 'LGSC', 'EC', 'HGSC', 'HGSC', 'MC', 'MC', 'LGSC', 'LGSC', 'MC', 'EC', 'CC', 'HGSC', 'CC', 'CC', 'CC', 'HGSC', 'CC', 'HGSC', 'MC', 'LGSC', 'MC', 'EC', 'CC', 'MC', 'MC', 'CC', 'CC', 'LGSC', 'EC', 'CC', 'HGSC', 'MC', 'LGSC', 'LGSC']
['MC', 'HGSC', 'HGSC', 'LGSC', 'LGSC', 'HGSC', 'EC', 'LGSC', 'HGSC', 'HGSC', 'EC', 'MC', 'LGSC', 'LGSC', 'EC', 'EC', 'CC', 'EC', 'EC', 'CC', 'CC', 'HGSC', 'CC', 'HGSC', 'MC', 'LGSC', 'MC', 'EC', 'CC', 'MC', 'MC', 'CC', 'CC', 'LGSC', 'EC', 'CC', 'HGSC', 'MC', 'LGSC', 'MC']


-------------------------

In [18]:
torch.save(model.state_dict(), 'models/anar2_best_model.pth')

In [21]:
model2 = AttentionMIL(input_dim=2048, hidden_dim=256, num_classes=num_classes)
model2.load_state_dict(torch.load('/root/ubc_ocean/anar/models/anar2_best_model.pth'))

<All keys matched successfully>